##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Автоматическое обновление кода до TensorFlow 2

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/upgrade">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    Смотрите на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/ru/guide/upgrade.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Запустите в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/ru/guide/upgrade.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    Изучайте код на GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs/site/ru/guide/upgrade.ipynb">
    <img src="https://www.tensorflow.org/images/download_logo_32px.png" />
    Скачайте ноутбук</a>
  </td>
</table>



Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

TensorFlow 2.0 включает много изменений API, таких как изменение порядка аргументов, переименование символов и изменение значений по умолчанию длы параметров. Ручное исправление всех этих модификций утомительно и подвержено ошибкам. Чтобы упростить изменения и сделать ваш переход на TF 2.0 как можно более плавным, команда TensorFlow создала утилиту `tf_upgrade_v2`, помогающую перейти от legacy кода к новому API.

Примечание: `tf_upgrade_v2` устанавливается автоматически для TensorFlow 1.13 и более поздних версий (включая все сборки TF 2.0).

Типичное использование выглядит так:

<pre class="devsite-terminal devsite-click-to-copy prettyprint lang-bsh">
tf_upgrade_v2 \
  --intree my_project/ \
  --outtree my_project_v2/ \
  --reportfile report.txt
</pre>

Это ускорит процесс обновления за счет конвертации существующих скриптов TensorFlow 1.x Python в TensorFlow 2.0.

Скрипт конвертации максимально автоматизирует процесс, но все еще существуют синтаксические и стилистические изменения, которые не могут быть выполнены скриптом.

## Модули совместимости

Некоторые символы API не могут быть обновлены просто с использованием замены строк. Чтобы гарантировать поддержку вашего кода в TensorFlow 2.0, скрипт обновления включает в себя модуль `compat.v1`. Этот модуль заменяет символы TF 1.x, такие как `tf.foo`, на эквивалентную ссылку` tf.compat.v1.foo`. Хотя модуль совместимости хорош, мы рекомендуем вам вручную вычитать замены и перенести их на новые API в пространстве имен `tf. *` вместо пространства имен `tf.compat.v1` как можно быстрее.

Из-за устаревания модулей TensorFlow 2.x (например, `tf.flags` и` tf.contrib`) некоторые изменения не могут быть обойдены путем переключения на `compat.v1`. Обновление этого кода может потребовать использования дополнительной библиотеки (например, [`absl.flags`] (https://github.com/abseil/abseil-py)) или переключения на пакет в [tenorflow / addons] (http: //www.github.com/tensorflow/addons).



## Рекомендуемый процесс обновления

Оставшаяся часть руководства демонстрирует использование скрипта обновления. Хоть скрипт обновления прост в использовании, очень рекомендуем вам использовать скрипт как часть следующего процесса: 

1. ** Модульное тестирование **: убедитесь, что в обновляемом коде имеется набор модульных тестов с разумным охватом. Это код Python, поэтому язык не защитит вас от многих классов ошибок. Также убедитесь, что все ваши зависимости были обновлены до совместимых с TensorFlow 2.0. 

1. **Установите TensorFlow 1.14**: Обновите ваш TensorFlow до последней версии TensorFlow 1.x, как минимум 1.14. Она включает финальный API TensorFlow 2.0 в `tf.compat.v2`.

1. **Протестируйте с 1.14**: Убедитесь, что ваши модульные тесты проходят на этом этапе. Вы будете повторно запускать их в процессе обновления поэтому важно начать с зеленого цвета.

1. **Запустите скрипт обновления**: Запустите `tf_upgrade_v2` на всем дереве исходного кода включая тесты. Это обновит ваш код до формата в котором он использует только символы доступные в TensorFlow 2.0. Устаревшие символы будут доступнны с `tf.compat.v1`. Это впоследствии потребует ручного внимания, но не сразу.

1. **Запустите ковертированные тесты с TensorFlow 1.14**: Ваш код должен все еще запускаться правильно в TensorFlow 1.14. Запустите снова модульные тесты. Любая ошибка в ваших тестах на этом этапе значит, что в скрипте обновления есть ошибка. [Сообщите нам пожалуйста об этом](https://github.com/tensorflow/tensorflow/issues).

1. **Проверьте отчет обновления на наличие предупреждений и ошибок**: Скрипт пишет файл отчета объясняющий все конвертации которые вам нужно перепроверить, или все действия которые нужно совершить вручную. Например: Любые оставшиеся экземпляры contrib требуют ручного удаления. Пожалуйста, обратитесь к [RFC для получения дополнительных инструкций](https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md). 

1. **Установите TensorFlow 2.0**: В этом месте переключение на TensorFlow 2.0 должно быть безопасно.

1. **Протестируйте с `v1.disable_v2_behavior`**: Re-running your tests with al `v1.disable_v2_behavior()` in the tests main function should give the same results as running under 1.14.

1. **Enable V2 Behavior**: Now that your tests work using the v2 API, you can start looking into turning on v2 behavior. Depending on how your code is written this may require some changes. See the [Migration guide](migrate.ipynb) for details.

## Using the upgrade script


### Setup

Before getting started ensure that TensorlFlow 2.0 is installed.

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [0]:
try:
  import tensorflow.compat.v2 as tf
except Exception:
  pass

tf.enable_v2_behavior()

print(tf.__version__)

Clone the [tensorflow/models](https://github.com/tensorflow/models) git repository so you have some code to test on:

In [0]:
!git clone --branch r1.13.0 --depth 1 https://github.com/tensorflow/models


### Read the help

The script should be installed with TensorFlow. Here is the builtin help:

In [0]:
!tf_upgrade_v2 -h

### Example TF1 code

Here is a simple TensorFlow 1.0 script:

In [0]:
!head -n 65 models/samples/cookbook/regression/custom_regression.py | tail -n 10

With TensorFlow 2.0 installed it does not run:

In [0]:
!(cd models/samples/cookbook/regression && python custom_regression.py)

### Single file

The upgrade script can be run on a single Python file:

In [0]:
!tf_upgrade_v2 \
  --infile models/samples/cookbook/regression/custom_regression.py \
  --outfile /tmp/custom_regression_v2.py

The script will print errors if it can not find a fix for the code. 

### Directory tree

Typical projects, including this simple example, will use much more than one file. Typically want to upgrade an entire package, so the script can also be run on a directory tree:

In [0]:
# upgrade the .py files and copy all the other files to the outtree
!tf_upgrade_v2 \
    --intree models/samples/cookbook/regression/ \
    --outtree regression_v2/ \
    --reportfile tree_report.txt

Note the one warning about the `dataset.make_one_shot_iterator` function.

Now the script works in with TensorFlow 2.0:

Note that because the `tf.compat.v1` module, the converted script will also run in TensorFlow 1.14. 

In [0]:
!(cd regression_v2 && python custom_regression.py 2>&1) | tail

## Detailed report

The script also reports a list of detailed changes. In this example it found one possibly unsafe transformation and included a warning at the top of the file: 

In [0]:
!head -n 20 tree_report.txt

Note again the one warning about the `Dataset.make_one_shot_iterator function`.

In other cases the output will explain the reasoning for non-trivial changes:

In [0]:
%%writefile dropout.py
import tensorflow as tf

d = tf.nn.dropout(tf.range(10), 0.2)
z = tf.zeros_like(d, optimize=False)

In [0]:
!tf_upgrade_v2 \
  --infile dropout.py \
  --outfile dropout_v2.py \
  --reportfile dropout_report.txt > /dev/null

In [0]:
!cat dropout_report.txt

Here is the modified file contents, note how the script adds argument names to deal with moved and renamed arguments:

In [0]:
!cat dropout_v2.py

A larger project might contain a few errors. For example convert the deeplab model:

In [0]:
!tf_upgrade_v2 \
    --intree models/research/deeplab \
    --outtree deeplab_v2 \
    --reportfile deeplab_report.txt > /dev/null

It produced the output files:

In [0]:
!ls deeplab_v2

But there were errors. The report will help you pin-point what you need to fix before this will run. Here are the first three errors:

In [0]:
!cat deeplab_report.txt | grep -i models/research/deeplab | grep -i error | head -n 3

## "Safety" mode

The conversion script also has a less invasive `SAFETY` mode that simply changes the imports to use the `tensorflow.compat.v1` module:

In [0]:
!cat dropout.py

In [0]:
!tf_upgrade_v2 --mode SAFETY --infile dropout.py --outfile dropout_v2_safe.py > /dev/null

In [0]:
!cat dropout_v2_safe.py

As you can see this doesn't upgrade your code, but does allow TensorFlow 1 code to run in TensorFlow 2


## Caveats

- Do not update parts of your code manually before running this script. In particular, functions that have had reordered arguments like `tf.argmax` or `tf.batch_to_space` cause the script to incorrectly add keyword arguments that mismap your existing code.

- The script assumes that `tensorflow` is imported using `import tensorflow as tf`.

- This script does not reorder arguments. Instead, the script adds keyword arguments to functions that have their arguments reordered.

- Check out [tf2up.ml](http://tf2up.ml) for a convenient tool to upgrade Jupyter
  notebooks and Python files in a GitHub repository.

To report upgrade script bugs or make feature requests, please file an issue on [GitHub](https://github.com/tensorflow/tensorflow/issues). And if you’re testing TensorFlow 2.0, we want to hear about it! Join the [TF 2.0 Testing community](https://groups.google.com/a/tensorflow.org/forum/#!forum/testing) and send questions and discussion to [testing@tensorflow.org](mailto:testing@tensorflow.org).